# Centralities (PageRank, Harmonic, Closeness) behaviour on Spotify Graph and SGC model

### Configuration steps

In [ ]:
import logging
import sys
import os
from featgraph.jwebgraph import start_jvm

jvm_path = None

logging.basicConfig(
  level=logging.INFO,
  format='%(asctime)s %(name)-12s %(levelname)-8s: %(message)s',
  datefmt='%Y-%m-%d %H:%M:%S',
)


do_install = True
package_root = ".."

if do_install:
  cmd = "{} -m pip install -U {} --use-feature=in-tree-build".format(
    sys.executable,
    package_root,
  )
  logging.info(cmd)
  os.system(cmd)
  cmd = "{} -m pip install -Ur {}".format(
    sys.executable,
    os.path.join(package_root, "notebooks-requirements.txt"),
  )
  logging.info(cmd)
  os.system(cmd)


start_jvm(jvm_path=jvm_path)

### Loading the Spotify graph

In [ ]:
# import Spotify graph
import featgraph.jwebgraph.utils
from featgraph import jwebgraph

spotify_basename = "graphs/spotify-2018"
graph = jwebgraph.utils.BVGraph(spotify_basename)
print(graph)

for r in (
  "graph", "properties", "ids.txt"
):
  if not os.path.isfile(graph.path(r)):
    raise FileNotFoundError(graph.path(r))

graph.reconstruct_offsets()
from featgraph.misc import pretty_print_int

print("Graph '{}' has\n{:>11} nodes\n{:>11} arcs".format(
  graph.basename,
  pretty_print_int(graph.numNodes()),
  pretty_print_int(graph.numArcs()),
))

### Creating an instance of the SGC model

In [ ]:
## create sgc graph in BVgraph format and store it
# from featgraph import sgc
# nx_basepath = "graph_SGC/SGCexampleNX"
# bv_basepath = "graph_SGC/SGCexampleBV"
# seed = 42
# model = sgc.SGCModel(
#     n_celeb=160,
#     n_leader=160,
#     n_masses=100000,
# )
# nxgraph = model(seed=seed)
# sgc.to_bv(nxgraph, bv_basepath)
# sgc_graph = jwebgraph.utils.BVGraph(bv_basepath)

### Loading the SGC graph

In [ ]:
# loading the SGC graph
import featgraph.jwebgraph.utils
from featgraph import jwebgraph

sgc_basename = "graph_SGC/SGCexampleBV"
sgc_graph = jwebgraph.utils.BVGraph(sgc_basename)
print(sgc_graph)

for r in (
  "graph", "properties"#, "ids.txt"
):
  if not os.path.isfile(sgc_graph.path(r)):
    raise FileNotFoundError(sgc_graph.path(r))

sgc_graph.reconstruct_offsets()
from featgraph.misc import pretty_print_int

print("Graph '{}' has\n{:>11} nodes\n{:>11} arcs".format(
  sgc_graph.basename,
  pretty_print_int(sgc_graph.numNodes()),
  pretty_print_int(sgc_graph.numArcs()),
))

# Test on Spotify graph

### Generating subgraphs for different popularity thresholds

In [ ]:
# Generating the subgraphs for different popularity thresholds
import numpy as np

thresh = np.arange(0, 81, 5)  # [0, 20, 30, 34, 35, 60]
type_filt = 'popularity'
dest_path = 'graphs/spotify-2018'
missing_value = -20
for t in thresh:  # per ogni soglia 
    print(t)
    # genero sottografo per soglia 
    subgraph_path = dest_path + '.mapped-' + type_filt + '-' + str(t)
    map_pop = list(map(lambda p: p > t, graph.popularity(missing_value)))
    subgraph = graph.transform_map(subgraph_path, map_pop, "spotify")
    # print("Subgraph generated")
    # print(subgraph.numNodes())

### Loading subgraphs for different popularity thresholds and computing mean centralities for supergenres

In [ ]:
import itertools
import numpy as np
from tqdm import tqdm

thresh = np.arange(0, 81, 5)  # [0, 20, 30, 34, 35, 60]
dest_path = 'graphs/spotify-2018'

hip_hop_cent = {'hc': [],
                'cc': [],
                'pr': []}
classical_cent = {'hc': [],
                  'cc': [],
                  'pr': []}
rock_cent = {'hc': [],
                'cc': [],
                'pr': []}

hip_hop_cent_n = {'hc': [],
                'cc': [],
                'pr': [],
                'pra': []}
classical_cent_n = {'hc': [],
                  'cc': [],
                  'pr': []}
rock_cent_n = {'hc': [],
                'cc': [],
                'pr': []}
hiphop_hc = []
hiphop_cc = []
hiphop_pr = []
classical_hc = []
classical_cc = []
classical_pr = []
rock_hc = []
rock_cc = []
rock_pr = []

for t in thresh:  # per ogni soglia
    # carico grafo
    basename = dest_path + '.mapped-' + 'popularity' + '-' + str(t)
    subgraph = jwebgraph.utils.BVGraph(basename)
    subgraph.reconstruct_offsets()
    
    # compute harmonic
    subgraph.compute_transpose()
    subgraph.compute_harmonicc()
    subgraph.compute_pagerank()
    subgraph.compute_closenessc()
    
    # select harmonic centralities for a specific genre: hip-hop
    map_gen = map(lambda p: 'hip-hop' in p, subgraph.supergenre())
    hc_gen = list(itertools.compress(subgraph.harmonicc(), map_gen))
    hc_gen_list = list(hc_gen)
    hip_hop_cent["hc"].append(sum(hc_gen_list)/len(hc_gen_list))
    hip_hop_cent_n["hc"].append((sum(hc_gen_list)/len(hc_gen_list))/subgraph.numNodes())
    hiphop_hc.append(hc_gen_list)

    # select harmonic centralities for a specific genre: classical
    map_gen = map(lambda p: 'classical' in p, subgraph.supergenre())
    hc_gen = list(itertools.compress(subgraph.harmonicc(), map_gen))
    hc_gen_list = list(hc_gen)
    classical_cent["hc"].append(sum(hc_gen_list)/len(hc_gen_list))
    classical_cent_n["hc"].append((sum(hc_gen_list)/len(hc_gen_list))/subgraph.numNodes())
    classical_hc.append(hc_gen_list)
    
    # select harmonic centralities for a specific genre: rock
    map_gen = map(lambda p: 'rock' in p, subgraph.supergenre())
    hc_gen = list(itertools.compress(subgraph.harmonicc(), map_gen))
    hc_gen_list = list(hc_gen)
    rock_cent["hc"].append(sum(hc_gen_list)/len(hc_gen_list))
    rock_cent_n["hc"].append((sum(hc_gen_list)/len(hc_gen_list))/subgraph.numNodes())
    rock_hc.append(hc_gen_list)
    
    # select closeness centralities for a specific genre: hip-hop
    map_gen = map(lambda p: 'hip-hop' in p, subgraph.supergenre())
    cc_gen = list(itertools.compress(subgraph.closenessc(), map_gen))
    cc_gen_list = list(cc_gen)
    hip_hop_cent["cc"].append(sum(cc_gen_list)/len(cc_gen_list))
    hip_hop_cent_n["cc"].append((sum(cc_gen_list)/len(cc_gen_list))/subgraph.numNodes())
    hiphop_cc.append(cc_gen_list)

    # select closeness centralities for a specific genre: classical
    map_gen = map(lambda p: 'classical' in p, subgraph.supergenre())
    cc_gen = list(itertools.compress(subgraph.closenessc(), map_gen))
    cc_gen_list = list(cc_gen)
    classical_cent["cc"].append(sum(cc_gen_list)/len(cc_gen_list))
    classical_cent_n["cc"].append((sum(cc_gen_list)/len(cc_gen_list))/subgraph.numNodes())
    classical_cc.append(cc_gen_list)
    
    # select closeness centralities for a specific genre: rock
    map_gen = map(lambda p: 'rock' in p, subgraph.supergenre())
    cc_gen = list(itertools.compress(subgraph.closenessc(), map_gen))
    cc_gen_list = list(cc_gen)
    rock_cent["cc"].append(sum(cc_gen_list)/len(cc_gen_list))
    rock_cent_n["cc"].append((sum(cc_gen_list)/len(cc_gen_list))/subgraph.numNodes())
    rock_cc.append(cc_gen_list)

    # select pagerank centralities for a specific genre: hip-hop
    map_gen = map(lambda p: 'hip-hop' in p, subgraph.supergenre())
    pr_gen = list(itertools.compress(subgraph.pagerank(), map_gen))
    pr_gen_list = list(pr_gen)
    hip_hop_cent["pr"].append(sum(pr_gen_list)/len(pr_gen_list))
    hip_hop_cent_n["pr"].append((sum(pr_gen_list)/len(pr_gen_list))/subgraph.numNodes())
    hiphop_pr.append(pr_gen_list)

    # select pagerank centralities for a specific genre: classical
    map_gen = map(lambda p: 'classical' in p, subgraph.supergenre())
    pr_gen = list(itertools.compress(subgraph.pagerank(), map_gen))
    pr_gen_list = list(pr_gen)
    classical_cent["pr"].append(sum(pr_gen_list)/len(pr_gen_list))
    classical_cent_n["pr"].append((sum(pr_gen_list)/len(pr_gen_list))/subgraph.numNodes())
    classical_pr.append(pr_gen_list)

    # select pagerank centralities for a specific genre: rock
    map_gen = map(lambda p: 'rock' in p, subgraph.supergenre())
    pr_gen = list(itertools.compress(subgraph.pagerank(), map_gen))
    pr_gen_list = list(pr_gen)
    rock_cent["pr"].append(sum(pr_gen_list)/len(pr_gen_list))
    rock_cent_n["pr"].append((sum(pr_gen_list)/len(pr_gen_list))/subgraph.numNodes())
    rock_pr.append(pr_gen_list)
    
    # select pagerank alpha
    #map_gen = map(lambda p: 'hip-hop' in p, subgraph.supergenre())
    #da = 0.1
    #alphas = np.linspace(da, 1, int(1/da - 1), endpoint=False)
    #for i, a in enumerate(tqdm(alphas)):
    #    subgraph.compute_pagerank(a)
    #    pra_gen = list(itertools.compress(subgraph.pagerank(a), map_gen))
    #    pra_gen_list = list(pr_gen)
    #    hip_hop_cent["pra"].append(sum(pra_gen_list)/len(pra_gen_list))
    #    hip_hop_cent_n["pra"].append((sum(pra_gen_list)/len(pra_gen_list))/subgraph.numNodes())

In [ ]:
import pygal
from IPython.display import SVG, display

for t in range(len(thresh)):
    box_plot = pygal.Box()
    box_plot.title = "Harmonic centrality: threshold on popularity > " + str(thresh[t])
    box_plot.add('Classical', classical_hc[t])
    box_plot.add('Hip-hop', hiphop_hc[t])
    box_plot.add('Rock', rock_hc[t])
    box_plot.render()
    #display(SVG(box_plot.render(disable_xml_declaration=True)))
    box_plot.render_to_png('fig_hc'+ str(t) +'.png') 

In [ ]:
for t in range(len(thresh)):
    box_plot = pygal.Box()
    box_plot.title = "Closeness centrality: threshold on popularity > " + str(thresh[t])
    box_plot.add('Classical', classical_cc[t])
    box_plot.add('Hip-hop', hiphop_cc[t])
    box_plot.add('Rock', rock_cc[t])
    box_plot.render()
    #display(SVG(box_plot.render(disable_xml_declaration=True)))
    box_plot.render_to_png('fig_cc'+ str(t) +'.png') 

In [ ]:
for t in range(len(thresh)):
    box_plot = pygal.Box()
    box_plot.title = "Pagerank centrality: threshold on popularity > " + str(thresh[t])
    box_plot.add('Classical', classical_pr[t])
    box_plot.add('Hip-hop', hiphop_pr[t])
    box_plot.add('Rock', rock_pr[t])
    box_plot.render()
    #display(SVG(box_plot.render(disable_xml_declaration=True)))
    box_plot.render_to_png('fig_pr'+ str(t) +'.png') 

In [ ]:
import matplotlib.pyplot as plt

plt.plot(thresh, hip_hop_cent['hc'], label='hip_hop')
plt.plot(thresh, classical_cent['hc'], label='classical')
plt.plot(thresh, rock_cent['hc'], label='rock')
plt.xlabel("Threshold on popularity")
plt.ylabel("Harmonic centrality")
plt.grid()
plt.legend()
plt.savefig("hc_spotify.png")
plt.show()

In [ ]:
plt.plot(thresh, hip_hop_cent_n['hc'], label='hip_hop')
plt.plot(thresh, classical_cent_n['hc'], label='classical')
plt.plot(thresh, rock_cent_n['hc'], label='rock')
plt.xlabel("Threshold on popularity")
plt.ylabel("Harmonic centrality")
plt.grid()
plt.legend()
plt.savefig("hc_norm_spotify.png")
plt.show()

In [ ]:
plt.plot(thresh, hip_hop_cent['cc'],label='hip_hop')
plt.plot(thresh, classical_cent['cc'], label='classical')
plt.plot(thresh, rock_cent['cc'], label='rock')
plt.xlabel("Threshold on popularity")
plt.ylabel("Closeness centrality")
plt.grid()
plt.legend()
plt.savefig("cc_spotify.png")
plt.show()

In [ ]:
plt.plot(thresh, hip_hop_cent_n['cc'],label='hip_hop')
plt.plot(thresh, classical_cent_n['cc'], label='classical')
plt.plot(thresh, rock_cent_n['cc'], label='rock')
plt.xlabel("Threshold on popularity")
plt.ylabel("Closeness centrality")
plt.grid()
plt.legend()
plt.savefig("cc_norm_spotify.png")
plt.show()

In [ ]:
plt.plot(thresh, hip_hop_cent['pr'],label='hip_hop')
plt.plot(thresh, classical_cent['pr'], label='classical')
plt.plot(thresh, rock_cent['pr'], label='rock')
plt.xlabel("Threshold on popularity")
plt.ylabel("Pagerank centrality")
plt.grid()
plt.legend()
plt.savefig("pr_spotify.png")
plt.show()

In [ ]:
plt.plot(thresh, hip_hop_cent_n['pr'],label='hip_hop')
plt.plot(thresh, classical_cent_n['pr'], label='classical')
plt.plot(thresh, rock_cent_n['pr'], label='rock')
plt.xlabel("Threshold on popularity")
plt.ylabel("Pagerank centrality")
plt.grid()
plt.legend()
plt.savefig("pr_norm_spotify.png")
plt.show()

# Generate subgraph from filtering by popularity 

### Filtering on SGC model

In [ ]:
# Generating the subgraphs for different popularity thresholds
import numpy as np

thresh = np.arange(0, 81, 5)  # [0, 20, 30, 34, 35, 60]
type_filt = 'popularity'
dest_path = 'graph_SGC/SGCexampleBV'
missing_value = -20

for t in thresh:  # per ogni soglia 
    # genero sottografo per soglia 
    sgc_subgraph_path = dest_path + '.mapped-' + type_filt + '-' + str(t)
    map_pop = list(map(lambda p: p > t, graph.popularity(missing_value)))
    sgc_subgraph = sgc_graph.transform_map(sgc_subgraph_path, map_pop, "sgc")
    print("Subgraph generated")
    # print(subgraph.numNodes())

In [ ]:
import itertools
import numpy as np
from tqdm import tqdm

thresh = np.arange(0, 81, 5)  # [0, 20, 30, 34, 35, 60]
dest_path = 'graph_SGC/SGCexampleBV'

masses_cent = {'hc': [],
                'cc': [],
                'pr': []}
celeb_cent = {'hc': [],
                  'cc': [],
                  'pr': []}
commlead_cent = {'hc': [],
                'cc': [],
                'pr': []}

masses_cent_n = {'hc': [],
                'cc': [],
                'pr': [],
                'pra': []}
celeb_cent_n = {'hc': [],
                  'cc': [],
                  'pr': []}
commlead_cent_n = {'hc': [],
                'cc': [],
                'pr': []}

masses_hc = []
masses_cc = []
masses_pr = []
celeb_hc = []
celeb_cc = []
celeb_pr = []
commlead_hc = []
commlead_cc = []
commlead_pr = []

for t in thresh:  # per ogni soglia
    print(t)
    # carico grafo
    basename = dest_path + '.mapped-' + 'popularity' + '-' + str(t)
    sgc_subgraph = jwebgraph.utils.BVGraph(basename)
    sgc_subgraph.reconstruct_offsets()
    
    # compute harmonic
    sgc_subgraph.compute_transpose()
    sgc_subgraph.compute_harmonicc()
    sgc_subgraph.compute_pagerank()
    sgc_subgraph.compute_closenessc()
    
    # select harmonic centralities for a specific type: masses
    map_type = map(lambda p: 'masses' in p, sgc_subgraph.type_sgc())
    hc_type = list(itertools.compress(sgc_subgraph.harmonicc(), map_type))
    hc_type_list = list(hc_type)
    if len(hc_type_list) > 0:
        masses_cent["hc"].append(sum(hc_type_list)/len(hc_type_list))
        masses_cent_n["hc"].append((sum(hc_type_list)/len(hc_type_list))/sgc_subgraph.numNodes())
    else:
        masses_cent["hc"].append(0)
        masses_cent_n["hc"].append(0)
    masses_hc.append(hc_type_list)
        
    
    # select harmonic centralities for a specific type: celeb
    map_type = map(lambda p: 'celeb' in p, sgc_subgraph.type_sgc())
    hc_type = list(itertools.compress(sgc_subgraph.harmonicc(), map_type))
    hc_type_list = list(hc_type)
    if len(hc_type_list) > 0:
        celeb_cent["hc"].append(sum(hc_type_list)/len(hc_type_list))
        celeb_cent_n["hc"].append((sum(hc_type_list)/len(hc_type_list))/sgc_subgraph.numNodes())
    else:
        celeb_cent["hc"].append(0)
        celeb_cent_n["hc"].append(0)
    celeb_hc.append(hc_type_list)
    
    # select harmonic centralities for a specific type: celeb
    map_type = map(lambda p: 'community leaders' in p, sgc_subgraph.type_sgc())
    hc_type = list(itertools.compress(sgc_subgraph.harmonicc(), map_type))
    hc_type_list = list(hc_type)
    if len(hc_type_list) > 0:
        commlead_cent["hc"].append(sum(hc_type_list)/len(hc_type_list))
        commlead_cent_n["hc"].append((sum(hc_type_list)/len(hc_type_list))/sgc_subgraph.numNodes())
    else:
        commlead_cent["hc"].append(0)
        commlead_cent_n["hc"].append(0)
    commlead_hc.append(hc_type_list)

    # select harmonic centralities for a specific type: masses
    map_type = map(lambda p: 'masses' in p, sgc_subgraph.type_sgc())
    cc_type = list(itertools.compress(sgc_subgraph.closenessc(), map_type))
    cc_type_list = list(cc_type)
    if len(cc_type_list) > 0:
        masses_cent["cc"].append(sum(cc_type_list)/len(cc_type_list))
        masses_cent_n["cc"].append((sum(cc_type_list)/len(cc_type_list))/sgc_subgraph.numNodes())
    else:
        masses_cent["cc"].append(0)
        masses_cent_n["cc"].append(0)
    masses_cc.append(cc_type_list)
    
    # select harmonic centralities for a specific type: celeb
    map_type = map(lambda p: 'celeb' in p, sgc_subgraph.type_sgc())
    cc_type = list(itertools.compress(sgc_subgraph.closenessc(), map_type))
    cc_type_list = list(cc_type)
    if len(cc_type_list) > 0:
        celeb_cent["cc"].append(sum(cc_type_list)/len(cc_type_list))
        celeb_cent_n["cc"].append((sum(cc_type_list)/len(cc_type_list))/sgc_subgraph.numNodes())
    else:
        celeb_cent["cc"].append(0)
        celeb_cent_n["cc"].append(0)
    celeb_cc.append(cc_type_list)
    
    # select harmonic centralities for a specific type: celeb
    map_type = map(lambda p: 'community leaders' in p, sgc_subgraph.type_sgc())
    cc_type = list(itertools.compress(sgc_subgraph.closenessc(), map_type))
    cc_type_list = list(cc_type)
    if len(cc_type_list) > 0:
        commlead_cent["cc"].append(sum(cc_type_list)/len(cc_type_list))
        commlead_cent_n["cc"].append((sum(cc_type_list)/len(cc_type_list))/sgc_subgraph.numNodes())
    else:
        commlead_cent["cc"].append(0)
        commlead_cent_n["cc"].append(0)
    commlead_cc.append(cc_type_list)
    
    # select harmonic centralities for a specific type: masses
    map_type = map(lambda p: 'masses' in p, sgc_subgraph.type_sgc())
    pr_type = list(itertools.compress(sgc_subgraph.pagerank(), map_type))
    pr_type_list = list(pr_type)
    if len(pr_type_list) > 0:
        masses_cent["pr"].append(sum(pr_type_list)/len(pr_type_list))
        masses_cent_n["pr"].append((sum(pr_type_list)/len(pr_type_list))/sgc_subgraph.numNodes())
    else:
        masses_cent["pr"].append(0)
        masses_cent_n["pr"].append(0)
    masses_pr.append(pr_type_list)
    
    # select harmonic centralities for a specific type: celeb
    map_type = map(lambda p: 'celeb' in p, sgc_subgraph.type_sgc())
    pr_type = list(itertools.compress(sgc_subgraph.pagerank(), map_type))
    pr_type_list = list(hc_type)
    if len(pr_type_list) > 0:
        celeb_cent["pr"].append(sum(pr_type_list)/len(pr_type_list))
        celeb_cent_n["pr"].append((sum(pr_type_list)/len(pr_type_list))/sgc_subgraph.numNodes())
    else:
        celeb_cent["pr"].append(0)
        celeb_cent_n["pr"].append(0)
    celeb_pr.append(pr_type_list)
    
    # select harmonic centralities for a specific type: celeb
    map_type = map(lambda p: 'community leaders' in p, sgc_subgraph.type_sgc())
    pr_type = list(itertools.compress(sgc_subgraph.pagerank(), map_type))
    pr_type_list = list(pr_type)
    if len(pr_type_list) > 0:
        commlead_cent["pr"].append(sum(pr_type_list)/len(pr_type_list))
        commlead_cent_n["pr"].append((sum(pr_type_list)/len(pr_type_list))/sgc_subgraph.numNodes())
    else:
        commlead_cent["pr"].append(0)
        commlead_cent_n["pr"].append(0)
    commlead_pr.append(pr_type_list)

In [ ]:
import pygal
from IPython.display import SVG, display

for t in range(len(thresh)):
    box_plot = pygal.Box()
    box_plot.title = "Harmonic centrality: threshold on popularity > " + str(thresh[t])
    box_plot.add('Community leaders', commlead_hc[t])
    box_plot.add('Celebrities', celeb_hc[t])
    box_plot.add('Masses', masses_hc[t])
    box_plot.render()
    #display(SVG(box_plot.render(disable_xml_declaration=True)))
    box_plot.render_to_png('figsgc_hc'+ str(t) +'.png') 

for t in range(len(thresh)):
    box_plot = pygal.Box()
    box_plot.title = "Closeness centrality: threshold on popularity > " + str(thresh[t])
    box_plot.add('Community leaders', commlead_cc[t])
    box_plot.add('Celebrities', celeb_cc[t])
    box_plot.add('Masses', masses_cc[t])
    box_plot.render()
    #display(SVG(box_plot.render(disable_xml_declaration=True)))
    box_plot.render_to_png('figsgc_cc'+ str(t) +'.png') 

for t in range(len(thresh)):
    box_plot = pygal.Box()
    box_plot.title = "Pagerank centrality: threshold on popularity > " + str(thresh[t])
    box_plot.add('Community leaders', commlead_pr[t])
    box_plot.add('Celebrities', celeb_pr[t])
    box_plot.add('Masses', masses_pr[t])
    box_plot.render()
    #display(SVG(box_plot.render(disable_xml_declaration=True)))
    box_plot.render_to_png('figsgc_pr'+ str(t) +'.png') 

In [ ]:
import matplotlib.pyplot as plt

plt.plot(thresh, celeb_cent['hc'], label='celeb')
plt.plot(thresh, commlead_cent['hc'], label='commlead')
plt.plot(thresh, masses_cent['hc'], label='masses')
plt.xlabel("Threshold on popularity")
plt.ylabel("Harmonic centrality")
plt.grid()
plt.legend()
plt.savefig("hc_sgc.png")
plt.show()

In [ ]:
plt.plot(thresh, celeb_cent_n['hc'], label='celeb')
plt.plot(thresh, commlead_cent_n['hc'], label='commlead')
plt.plot(thresh, masses_cent_n['hc'], label='masses')
plt.xlabel("Threshold on popularity")
plt.ylabel("Harmonic centrality")
plt.grid()
plt.legend()
plt.savefig("hc_norm_sgc.png")
plt.show()

In [ ]:
plt.plot(thresh, celeb_cent['cc'], label='celeb')
plt.plot(thresh, commlead_cent['cc'], label='commlead')
plt.plot(thresh, masses_cent['cc'], label='masses')
plt.xlabel("Threshold on popularity")
plt.ylabel("Closeness centrality")
plt.grid()
plt.legend()
plt.savefig("cc_sgc.png")
plt.show()

In [ ]:
plt.plot(thresh, celeb_cent_n['cc'], label='celeb')
plt.plot(thresh, commlead_cent_n['cc'], label='commlead')
plt.plot(thresh, masses_cent_n['cc'], label='masses')
plt.xlabel("Threshold on popularity")
plt.ylabel("Closeness centrality")
plt.grid()
plt.legend()
plt.savefig("cc_norm_sgc.png")
plt.show()

In [ ]:
plt.plot(thresh, celeb_cent['pr'], label='celeb')
plt.plot(thresh, commlead_cent['pr'], label='commlead')
plt.plot(thresh, masses_cent['pr'], label='masses')
plt.xlabel("Threshold on popularity")
plt.ylabel("Pagerank centrality")
plt.grid()
plt.legend()
plt.savefig("pr_sgc.png")
plt.show()

In [ ]:
plt.plot(thresh, celeb_cent_n['pr'], label='celeb')
plt.plot(thresh, commlead_cent_n['pr'], label='commlead')
plt.plot(thresh, masses_cent_n['pr'], label='masses')
plt.xlabel("Threshold on popularity")
plt.ylabel("Pagerank centrality")
plt.grid()
plt.legend()
plt.savefig("pr_norm_sgc.png")
plt.show()